# Creating House of PlantAgent

## Housekeeping Imports

In [59]:
import os
from typing import Annotated, Callable, Awaitable, List, Dict, Any, Optional
from dotenv import load_dotenv 
from langchain.agents import create_agent
from dataclasses import dataclass
from langchain.tools import tool, ToolRuntime
from langchain.chat_models import init_chat_model
from dataclasses import dataclass
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents.structured_output import ToolStrategy
import getpass
import json
import pprint, json, dataclasses
from collections.abc import Mapping

# For web-searching tool
from langchain.tools import tool
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper

load_dotenv() # Actually loading files from .env


True

## Loading API KEY

In [49]:
if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

## System Prompt

In [50]:
SYSTEM_PROMPT = """You are an expert vegan/plant-based nutritionist and meal generator.
Your name is House of PlantAgent.

You have access to one tool, which you MUST use before generating each response:

- search_plantbased_recipe: use this to search vegan recipe websites based on the user's stated preferences.
"""

## Defining Tool

In [51]:
@tool
def search_plantbased_recipe(query:str) -> str:
    """
    Safe wrapper that always returns a string.
    Use this as the tool passed to the agent.

    Argument 'query': The search query string"""
    try:
        wrapper = DuckDuckGoSearchAPIWrapper()
        try:
            # prefer .run()
            result = wrapper.run(query)
        except Exception:
            # fallback if the wrapper method differs
            result = wrapper.invoke(query)

        # If result is already a string, return it.
        if isinstance(result, str):
            return result
        # If it's a dict/list/obj, try to serialise to json as safe fallback
        try:
            return json.dumps(result, default=str, ensure_ascii=False)
        except Exception:
            return str(result)
    except Exception as e:
        # Always return a string without raising so the agent gets a tool response
        return f"ERROR in search_plantbased_recipe: {type(e).__name__}: {e}"


@dataclass # What does all this do again?
class Context:
    """Custom runtime context schema."""
    user_id: str

### Testing Tool

In [52]:
out = search_plantbased_recipe.invoke("tofu and noodles recipe")
print(type(out), repr(out)[:200])

<class 'str'> 'Tofu Noodles Recipe ... Tofu And Snow Pea Noodle Bowl ... Rice Noodle Bowl With Peanut Sauce And Roasted Tof... Tofu shirataki noodles are made from tofu and konjac yam root, which means they’re low 


## Loading Model

In [53]:
model = init_chat_model(
    "gpt-4.1",
    temperature=0.5,
    timeout=10,
    max_tokens=500
)

### Defining Response Format

In [54]:
# Not entirely sure how or if this is required
# We use a dataclass here, but Pydantic models are also supported.
# @dataclass
# class ResponseFormat:
#     """Response schema for the agent."""
#     # A punny response (always required)
#     from_source: str
#     # Any interesting information about the weather if available
#     weather_conditions: str | None = None

### Adding Memory

In [55]:
checkpointer = InMemorySaver()

### Testing a Run

In [60]:
agent = create_agent(
    model=model,
    system_prompt=SYSTEM_PROMPT,
    tools=[search_plantbased_recipe],
    context_schema=Context,
    # response_format=ToolStrategy(ResponseFormat),
    checkpointer=checkpointer
)

# `thread_id` is a unique identifier for a given conversation.
config = {"configurable": {"thread_id": "1"}}

response = agent.invoke(
    {"messages": [{"role": "user", "content": "Search me a quick tofu and noodles recipe."}]},
    config=config,
    context=Context(user_id="1")
)

print("type(response) =", type(response))
if isinstance(response, dict):
    print("top-level keys:", list(response.keys()))
    pprint.pprint({k: type(v) for k, v in response.items()})
# ResponseFormat(
#     punny_response="Florida is still having a 'sun-derful' day! The sunshine is playing 'ray-dio' hits all day long! I'd say it's the perfect weather for some 'solar-bration'! If you were hoping for rain, I'm afraid that idea is all 'washed up' - the forecast remains 'clear-ly' brilliant!",
#     weather_conditions="It's always sunny in Florida!"
# )


# # Note that we can continue the conversation using the same `thread_id`.
# response = agent.invoke(
#     {"messages": [{"role": "user", "content": "thank you!"}]},
#     config=config,
#     context=Context(user_id="1")
# )

# print(response['structured_response'])
# # ResponseFormat(
# #     punny_response="You're 'thund-erfully' welcome! It's always a 'breeze' to help you stay 'current' with the weather. I'm just 'cloud'-ing around waiting to 'shower' you with more forecasts whenever you need them. Have a 'sun-sational' day in the Florida sunshine!",
# #     weather_conditions=None
# )

type(response) = <class 'dict'>
top-level keys: ['messages']
{'messages': <class 'list'>}


## Initial Output

Tofu Soba Noodles - This quick and easy vegetarian noodle dish comes together in just 20 minutes! Here are 50+ beginner friendly, easy tofu recipes to make for dinner tonight! These quick and easy tofu recipes range from rice dishes, stir fries, noodles , sandwiches and more! These tofu noodles are so quick and easy you won't even believe it. Besides you need just a handful of ingredients you probably already have. Here's what to get to make this yum vegan lunch! Keywords: tofu noodles recipe, mushroom broth with tofu , gluten free noodle dish, low carb tofu noodles , high protein vegan meals, quick mushroom broth recipe, vegan noodle alternatives, easy tofu recipes, healthy noodle options, fresh tofu skin. Quick creamy tofu noodles . Based on 10 ratings.Drain the noodles , reserving about 100 ml of the cooking water. Return the noodles to the hot pot and mix with the tofu sauce, adding the noodle water little by little as needed. The sauce should be creamy but not too runny.

{'messages': [HumanMessage(content='Search me a quick tofu and noodles recipe.', additional_kwargs={}, response_metadata={}, id='a7f6a91f-43bf-4e78-852f-721b47848e7b'), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 117, 'total_tokens': 139, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_09249d7c7b', 'id': 'chatcmpl-CiTRUpVD1wQUd7YwXqcLAanwC7RBj', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--9a38e88d-0db9-4996-9f07-d139272e2e05-0', tool_calls=[{'name': 'search_plantbased_recipe', 'args': {'user_preferences': 'quick tofu and noodles recipe'}, 'id': 'call_HnbJ4Mf8sdP72DSFb5xQ74cb', 'type': 'tool_call'}], usage_metadata={'input_tokens': 117, 'output_tokens': 22, 'total_tokens': 139, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}), ToolMessage(content='null', name='search_plantbased_recipe', id='578e76d2-db94-4a10-b558-daa93436d53b', tool_call_id='call_HnbJ4Mf8sdP72DSFb5xQ74cb'), AIMessage(content="I couldn't find a specific recipe at the moment, but I can suggest a quick tofu and noodles dish:\n\n- Stir-fry cubed tofu until golden.\n- Add cooked noodles (rice or wheat), your favorite veggies (like bell peppers, broccoli, or snap peas), and a simple sauce (soy sauce, garlic, ginger, and a dash of sesame oil).\n- Toss everything together for a few minutes until heated through.\n\nIf you’d like a detailed recipe or specific flavors (like spicy, peanut, or teriyaki), let me know and I’ll refine the search!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 117, 'prompt_tokens': 151, 'total_tokens': 268, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_09249d7c7b', 'id': 'chatcmpl-CiTRYQK51YwJ4ygaVnh5IjUx9tk1e', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--3707c1e0-56dc-45bd-8640-389aa131649b-0', usage_metadata={'input_tokens': 151, 'output_tokens': 117, 'total_tokens': 268, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}), HumanMessage(content='Search me a quick tofu and noodles recipe.', additional_kwargs={}, response_metadata={}, id='534b04a2-e031-4c44-8d52-a48294d96e30'), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 284, 'total_tokens': 305, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_09249d7c7b', 'id': 'chatcmpl-CiTSn2P1dyNp5y1C3kUSZLi22Fykj', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--78a39263-ec4e-4e0e-86a0-5671c45ba852-0', tool_calls=[{'name': 'search_plantbased_recipe', 'args': {'user_preferences': 'quick tofu and noodles'}, 'id': 'call_fPIQsgSBchhpkE6BhAtXmnpB', 'type': 'tool_call'}], usage_metadata={'input_tokens': 284, 'output_tokens': 21, 'total_tokens': 305, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}), ToolMessage(content='null', name='search_plantbased_recipe', id='615ae91d-0823-438f-b1ec-f9324fb317ae', tool_call_id='call_fPIQsgSBchhpkE6BhAtXmnpB'), AIMessage(content="I wasn't able to retrieve a specific recipe at this moment. However, I can guide you on making a quick tofu and noodles meal:\n\n1. Cook your favorite noodles (rice, soba, or wheat noodles) according to package instructions.\n2. While the noodles cook, cube firm tofu and pan-fry until golden on all sides.\n3. Add sliced veggies (like bell pepper, carrot, or snap peas) to the pan and stir-fry for a few minutes.\n4. Mix in the cooked noodles and toss with a sauce made from soy sauce, garlic, ginger, and a touch of maple syrup or sesame oil.\n5. Serve hot and garnish with green onions or sesame seeds.\n\nLet me know if you want a specific cuisine or flavor profile (spicy, peanut, teriyaki, etc.), and I’ll tailor the recipe further for you!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 175, 'prompt_tokens': 317, 'total_tokens': 492, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_433e8c8649', 'id': 'chatcmpl-CiTSpqr13gAINQ7QpWjBQivCV8rJz', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--b95b8e1a-dbf1-438f-8528-16160b346978-0', usage_metadata={'input_tokens': 317, 'output_tokens': 175, 'total_tokens': 492, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}